### 1. Importing Libraries

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import urllib.request # library to open URLs

!pip -q install bs4
from bs4 import BeautifulSoup # import the BeautifulSoup library so we can parse HTML and XML documents

import json # library to handle JSON files

!pip -q install geopy #install geopy
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
#RateLimiter, class which can be used to automatically add delays between geocoding calls to reduce the load on the Geocoding service. 
#Also it can retry failed requests and swallow errors for individual rows.
from geopy.extra.rate_limiter import RateLimiter 

import requests # library to handle requests

print('Libraries imported.')

Libraries imported.


In [2]:
#Bokeh and dependency libraries
!pip -q install bokeh
!pip -q install Jinja2 >=2.7
!pip -q install numpy >=1.11.3
!pip -q install packaging >=16.8
!pip -q install pillow >=4.0
!pip -q install python-dateutil >=2.1
!pip -q install PyYAML >=3.10
!pip -q install six >=1.5.2
!pip -q install tornado >=5
!pip -q install typing_extensions >=3.7.4

### 2. Get Toronto Dataset

##### Scrape Toronto Data from Wikipedia page.

In [3]:
# specify the URL/web page we are going to be scraping
url = "https://en.wikipedia.org/w/index.php?title=List_of_postal_codes_of_Canada:_M&oldid=945633050"
# open the url using urllib.request and put the HTML into the page variable
page = urllib.request.urlopen(url)
# parse the HTML from our URL into the BeautifulSoup parse tree format
soup = BeautifulSoup(page)
print('Soup ready!')

Soup ready!


In [ ]:
# analyse the HTML underlying the website
#print(soup.prettify())

In [ ]:
# use the 'find_all' function to bring back all instances of the 'table' tag in the HTML and store in 'all_tables' variable
all_tables=soup.find_all("table")
#all_tables

In [4]:
right_table=soup.find('table', class_='wikitable sortable')
#right_table

# Loop through the rows

A=[]
B=[]
C=[]

for row in right_table.findAll('tr'):
    cells=row.findAll('td')
    if len(cells)==3:
        A.append(cells[0].find(text=True))
        B.append(cells[1].find(text=True))
        C.append(cells[2].find(text=True))

In [5]:
#convert list to dataframe
df=pd.DataFrame(A,columns=['PostalCode'])
df['Borough']=B
df['Neighborhood']=C
df.head()

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


##### Explore Dataset and Data Cleaning.

In [6]:
print('Dataframe shape {}'.format(df.shape))
df.info()

Dataframe shape (287, 3)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 287 entries, 0 to 286
Data columns (total 3 columns):
PostalCode      287 non-null object
Borough         287 non-null object
Neighborhood    287 non-null object
dtypes: object(3)
memory usage: 6.8+ KB


In [7]:
print('Total Boroughs {}'.format(df['Borough'].drop_duplicates().reset_index(drop=True).count()))
df["Borough"].value_counts()

Total Boroughs 11


Not assigned        77
Etobicoke           45
North York          38
Scarborough         37
Downtown Toronto    37
Central Toronto     17
West Toronto        13
York                 9
East Toronto         7
East York            6
Mississauga          1
Name: Borough, dtype: int64

In [8]:
#Remove 'PostalCode' column.
df = df.drop(['PostalCode'],axis=1)

#Remove rows with Borough equals to 'Not assigned' and 'mississuaga'.
df = df[(df.Borough != 'Mississauga') & (df.Borough != 'Not assigned')].reset_index(drop=True)

print('Total Borough {}'.format(df['Borough'].drop_duplicates().reset_index(drop=True).count()))
print('Total Neighborhood {}'.format(df['Neighborhood'].count()))
print('Dataframe shape {}'.format(df.shape))
df.head()

Total Borough 9
Total Neighborhood 209
Dataframe shape (209, 2)


,Borough,Neighborhood
0,North York,Parkwoods
1,North York,Victoria Village
2,Downtown Toronto,Harbourfront
3,North York,Lawrence Heights
4,North York,Lawrence Manor


##### Get Neighborhood Latitude and Longitude using Geopy.

In [9]:
geolocator = Nominatim(user_agent="tor_explorer")
geocode = RateLimiter(geolocator.geocode, min_delay_seconds=1)

# create empty lists
list_lat = []   
list_lng = []

for index, row in df.iterrows(): # iterate over rows in dataframe

    Neighborhood = row['Neighborhood']
    City = 'Toronto'     
    address = str(Neighborhood)+', '+ City
    #print(address)

    #location = geolocator.geocode(address)
    location = geocode(address)
    if location is None :
        lat = None
        lng = None
    else:
        lat = location.latitude
        lng = location.longitude
    #print('{},{}'.format(lat,lng))

    list_lat.append(lat)
    list_lng.append(lng)

# create new columns from lists    
df['Latitude'] = list_lat   
df['Longitude'] = list_lng

print('Dataframe shape {}'.format(df.shape))
df.head()

Dataframe shape (209, 4)


,Borough,Neighborhood,Latitude,Longitude
0,North York,Parkwoods,43.758800,-79.320197
1,North York,Victoria Village,43.732658,-79.311189
2,Downtown Toronto,Harbourfront,43.640080,-79.380150
3,North York,Lawrence Heights,43.722778,-79.450933
4,North York,Lawrence Manor,43.722079,-79.437507


In [10]:
#remove rows with NaN values
toronto_data = df.dropna().reset_index(drop=True)
print('Dataframe shape {}'.format(toronto_data.shape))
toronto_data.head()

Dataframe shape (201, 4)


,Borough,Neighborhood,Latitude,Longitude
0,North York,Parkwoods,43.758800,-79.320197
1,North York,Victoria Village,43.732658,-79.311189
2,Downtown Toronto,Harbourfront,43.640080,-79.380150
3,North York,Lawrence Heights,43.722778,-79.450933
4,North York,Lawrence Manor,43.722079,-79.437507


### 3. Explore Toronto Neighborhood 

##### Count Neighborhood of each Borough.

In [11]:
toronto_data['Borough'].value_counts()

Etobicoke           43
North York          37
Scarborough         37
Downtown Toronto    36
Central Toronto     17
West Toronto        13
East York            6
East Toronto         6
York                 6
Name: Borough, dtype: int64

In [12]:
# The code was removed by Watson Studio for sharing.

##### Search Mosques around Toronto Neighborhood using Foursquare API.

In [13]:
def getNearbyMosques(boroughs,neighborhoods, latitudes, longitudes):
    
    venues_list=[]
    for borough, neighborhood, lat, lng in zip(boroughs, neighborhoods, latitudes, longitudes):
        #print(neighborhoods)
            
        # create the API request URL
        LIMIT = 100 # limit of number of venues returned by Foursquare API
        radius = 10000 # define radius
        search_query = 'mosque'
        url = 'https://api.foursquare.com/v2/venues/search?&client_id={}&client_secret={}&v={}&ll={},{}&query={}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET,
            VERSION, 
            lat, 
            lng, 
            search_query,
            radius, 
            LIMIT)
        #print(url)
         # make the GET request
        results = requests.get(url).json()["response"]['venues']
        
        #print(result)
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            borough,
            neighborhood, 
            lat, 
            lng, 
            v['name'], 
            v['location']['lat'], 
            v['location']['lng']) for v in results])

    nearby_mosques = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_mosques.columns = ['Boroughs',
                             'Neighborhoods',
                             'N_Lat',
                             'N_Lng',
                             'Mosques',
                             'M_Lat',
                             'M_Lng'] 
                  
    
    return(nearby_mosques)

In [14]:
toronto_mosques = getNearbyMosques(
    boroughs= toronto_data['Borough'],
    neighborhoods= toronto_data['Neighborhood'],
    latitudes= toronto_data['Latitude'],
    longitudes= toronto_data['Longitude']
    )

In [15]:
toronto_mosques = toronto_mosques.drop_duplicates(subset='Mosques', keep="last")
print(toronto_mosques.shape)
print('toronto_mosques shape {}'.format(toronto_mosques.shape))
toronto_mosques.head(15)

(23, 7)
toronto_mosques shape (23, 7)


,Boroughs,Neighborhoods,N_Lat,N_Lng,Mosques,M_Lat,M_Lng
362,North York,York University,43.779242,-79.483559,Baitul Islam Mosque,43.865999,-79.534002
840,Central Toronto,North Midtown,43.162128,-77.619604,Hamidiye Mosque,43.160714,-77.566286
1080,Downtown Toronto,Railway Lands,49.266236,-123.237060,Ajyal Mosque | مسجد أجيال (Ajyal Mosque),49.279837,-123.108489
1081,Downtown Toronto,South Niagara,43.072145,-79.073868,Mosque Aisha,43.094220,-79.084564
1123,Etobicoke,Thistletown,43.737266,-79.565317,Vaughn Islamic Community Centre Mosque,43.828300,-79.537521
1126,Etobicoke,Thistletown,43.737266,-79.565317,Masjid E Saliheen,43.705980,-79.644635
1158,Scarborough,Upper Rouge,43.804930,-79.165837,Nugget mosque,43.780834,-79.232931
1159,Scarborough,Upper Rouge,43.804930,-79.165837,Jame Masjid Markham,43.841908,-79.264350
1180,Downtown Toronto,Underground city,43.770145,-79.374863,Abu Huraira Mosque Inc (Abu Huraira Center),43.772896,-79.334384
1183,Downtown Toronto,Underground city,43.770145,-79.374863,Süleymaniye Mosque Finch,43.763820,-79.482940


In [16]:
mosque_count = pd.DataFrame(toronto_mosques['Neighborhoods'].value_counts())
mosque_count = mosque_count.reset_index()
mosque_count.rename(columns={'index':'Neighborhoods','Neighborhoods':'NoofMosque'},inplace = True)
mosque_count

,Neighborhoods,NoofMosque
0,South of Bloor,8
1,Underground city,3
2,Royal York South West,3
3,Upper Rouge,2
4,Thistletown,2
5,The Queensway East,1
6,York University,1
7,Railway Lands,1
8,North Midtown,1
9,South Niagara,1


In [17]:
#merge dataframe
mosque_count = toronto_mosques.merge(mosque_count).drop(['N_Lat','N_Lng','Mosques','M_Lat','M_Lng'], axis=1)
mosque_count.head()

,Boroughs,Neighborhoods,NoofMosque
0,North York,York University,1
1,Central Toronto,North Midtown,1
2,Downtown Toronto,Railway Lands,1
3,Downtown Toronto,South Niagara,1
4,Etobicoke,Thistletown,2


##### Search Halal Restaurants Nearby Mosques using Foursquare API.

In [18]:
def getNearbyRestaurants(boroughs,neighborhoods, mosques, latitudes, longitudes):
    
    venues_list=[]
    for borough, neighborhood, mosque, lat, lng in zip(boroughs, neighborhoods, mosques, latitudes, longitudes):
                    
        # create the API request URL
        LIMIT = 100 # limit of number of venues returned by Foursquare API
        radius = 500 # define radius
        search_query = 'Halal Restaurant'
        url = 'https://api.foursquare.com/v2/venues/search?&client_id={}&client_secret={}&v={}&ll={},{}&query={}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            search_query,
            radius, 
            LIMIT)
            
         # make the GET request
        results = requests.get(url).json()["response"]['venues']
        #print(results)
        
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            borough,
            neighborhood,
            mosque,
            lat, 
            lng, 
            v['name'], 
            v['location']['lat'], 
            v['location']['lng']) for v in results])  

    nearby_resturants = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_resturants.columns = ['Boroughs', 
                                 'Neighborhoods',
                  'Mosques',
                  'M_Lat', 
                  'M_Lng', 
                  'Halal Restaurants', 
                  'R_Lat', 
                  'R_Lng'] 
                    
    return(nearby_resturants)

In [19]:
toronto_halal = getNearbyRestaurants(
    boroughs= toronto_mosques['Boroughs'],
    neighborhoods= toronto_mosques['Neighborhoods'],
    mosques= toronto_mosques['Mosques'],
    latitudes= toronto_mosques['M_Lat'],
    longitudes= toronto_mosques['M_Lng']
    )

In [20]:
toronto_halal = toronto_halal.drop_duplicates(subset='Halal Restaurants', keep="last")
print(toronto_halal.shape)
toronto_halal.head()

(130, 8)


,Boroughs,Neighborhoods,Mosques,M_Lat,M_Lng,Halal Restaurants,R_Lat,R_Lng
0,Central Toronto,North Midtown,Hamidiye Mosque,43.160714,-77.566286,Carroll's,43.160017,-77.563975
1,Central Toronto,North Midtown,Hamidiye Mosque,43.160714,-77.566286,Chang Lung Chinese Restaurant,43.163400,-77.563753
2,Central Toronto,North Midtown,Hamidiye Mosque,43.160714,-77.566286,Jamaican Restaurant,43.160168,-77.570656
3,Central Toronto,North Midtown,Hamidiye Mosque,43.160714,-77.566286,"Win Wah Chinese Restaurant, Rochester",43.159832,-77.559790
4,Downtown Toronto,Railway Lands,Ajyal Mosque | مسجد أجيال (Ajyal Mosque),49.279837,-123.108489,Yagger's Downtown Restaurant & Sports Bar,49.283161,-123.112503


In [21]:
halal_count = pd.DataFrame(toronto_halal['Mosques'].value_counts())
halal_count = halal_count.reset_index()
halal_count.rename(columns={'index':'Mosques','Mosques':'NoofHalalRestaurant'}, inplace = True)
halal_count

,Mosques,NoofHalalRestaurant
0,Toronto Mosque,38
1,Ajyal Mosque | مسجد أجيال (Ajyal Mosque),30
2,Hamza Mosque,9
3,Jame Abu Bakr Siddique Masjid,7
4,Mosque Aisha,6
5,Baitul Aman Mosque,6
6,Mosque,5
7,Masjid E Saliheen,5
8,Süleymaniye Mosque Finch,5
9,Vaughn Islamic Community Centre Mosque,4


In [22]:
#merge dataframe
halal_count = toronto_halal.merge(halal_count).drop(['M_Lat','M_Lng','Halal Restaurants','R_Lat','R_Lng'], axis=1)
halal_count.head()

,Boroughs,Neighborhoods,Mosques,NoofHalalRestaurant
0,Central Toronto,North Midtown,Hamidiye Mosque,4
1,Central Toronto,North Midtown,Hamidiye Mosque,4
2,Central Toronto,North Midtown,Hamidiye Mosque,4
3,Central Toronto,North Midtown,Hamidiye Mosque,4
4,Downtown Toronto,Railway Lands,Ajyal Mosque | مسجد أجيال (Ajyal Mosque),30


##### Search markets, groceries store, etc nearby Mosque around Neighborhood using Foursquare API.

In [23]:
def getNearbyMarkets(boroughs, neighborhoods, mosques, latitudes, longitudes):
    
    markets_list=[]
    for borough, neighborhood, mosque, lat, lng in zip(boroughs, neighborhoods, mosques, latitudes, longitudes):
    
            
        # create the API request URL
        LIMIT = 100 # limit of number of venues returned by Foursquare API
        radius = 500 # define radius
        search_query = 'Market'
        url = 'https://api.foursquare.com/v2/venues/search?&client_id={}&client_secret={}&v={}&ll={},{}&query={}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            search_query,
            radius, 
            LIMIT)
        #print(url)
        
            
         # make the GET request
        results = requests.get(url).json()["response"]['venues']
        #print(results)

        # return only relevant information for each nearby venue
        markets_list.append([(
            borough, 
            neighborhood,
            mosque,
            lat, 
            lng, 
            v['name'], 
            v['location']['lat'], 
            v['location']['lng'])for v in results])
       
    
    nearby_markets = pd.DataFrame([item for market_list in markets_list for item in market_list])
    nearby_markets.columns = [
        'Boroughs',
        'Neighborhoods',
        'Mosques',
        'M_Lat',
        'M_Lng',
        'Markets',
        'Mr_Lat',
        'Mr_Lng'] 
                  
    
    return(nearby_markets)

In [24]:
toronto_market = getNearbyMarkets(
    boroughs= toronto_mosques['Boroughs'],
    neighborhoods= toronto_mosques['Neighborhoods'],
    mosques= toronto_mosques['Mosques'],
    latitudes= toronto_mosques['M_Lat'],
    longitudes= toronto_mosques['M_Lng']
    )

In [25]:
toronto_market = toronto_market.drop_duplicates(subset='Markets', keep="last")
print(toronto_market.shape)
toronto_market.head()

(97, 8)


,Boroughs,Neighborhoods,Mosques,M_Lat,M_Lng,Markets,Mr_Lat,Mr_Lng
0,Central Toronto,North Midtown,Hamidiye Mosque,43.160714,-77.566286,The Insurance Market Place,43.162029,-77.565576
1,Central Toronto,North Midtown,Hamidiye Mosque,43.160714,-77.566286,Ariana Super Market,43.164597,-77.563972
2,Central Toronto,North Midtown,Hamidiye Mosque,43.160714,-77.566286,Cal's Liquor,43.164074,-77.564241
3,Downtown Toronto,Railway Lands,Ajyal Mosque | مسجد أجيال (Ajyal Mosque),49.279837,-123.108489,Nesters Market,49.282677,-123.107602
4,Downtown Toronto,Railway Lands,Ajyal Mosque | مسجد أجيال (Ajyal Mosque),49.279837,-123.108489,Eastside Artisan Market,49.282750,-123.108185


In [26]:
market_count = pd.DataFrame(toronto_market['Neighborhoods'].value_counts())
market_count = market_count.reset_index()
market_count.rename(columns={'index':'Neighborhoods','Neighborhoods':'NoofMarket'}, inplace = True)
market_count

,Neighborhoods,NoofMarket
0,South of Bloor,53
1,Railway Lands,31
2,Underground city,3
3,North Midtown,3
4,Thistletown,3
5,Upper Rouge,2
6,South Niagara,1
7,Royal York South West,1


In [27]:
#merge dataframe
market_count = toronto_market.merge(market_count).drop(['Mosques','M_Lat','M_Lng','Markets','Mr_Lat','Mr_Lng'], axis=1)
market_count.head()

,Boroughs,Neighborhoods,NoofMarket
0,Central Toronto,North Midtown,3
1,Central Toronto,North Midtown,3
2,Central Toronto,North Midtown,3
3,Downtown Toronto,Railway Lands,31
4,Downtown Toronto,Railway Lands,31


In [28]:
toronto_market.groupby(['Neighborhoods', 'Mosques']).agg(['count'])

Boroughs  \
                                                                             count   
Neighborhoods          Mosques                                                       
North Midtown                                                                    3   
         Hamidiye Mosque                                                        31   
Railway Lands          Ajyal Mosque | مسجد أجيال (Ajyal Mosque)                  1   
Royal York South West                                                            1   
 Baitul Hamd Mosque                                                              3   
South Niagara          Mosque Aisha                                              2   
South of Bloor                                                                   4   
        Albanian Mosque - Albanian Muslim Society of To...                      10   
                       Baitul Aman Mosque                                        1   
                       Bosnian Mosque Birmingham                                33   
                       Hamza Mosque                                              3   
                       Omar Bin Khattab Mosque                                   3   
                       Toronto Mosque                                            2   
Thistletown            Vaughn Islamic Community Centre Mosque                        
Underground city       Abu Huraira Mosque Inc (Abu Huraira Center)                   
Upper Rouge            Nugget mosque                                                 

                                                                          M_Lat  \
                                                                          count   
Neighborhoods          Mosques                                                    
North Midtown                                                                 3   
         Hamidiye Mosque                                                     31   
Railway Lands          Ajyal Mosque | مسجد أجيال (Ajyal Mosque)               1   
Royal York South West                                                         1   
 Baitul Hamd Mosque                                                           3   
South Niagara          Mosque Aisha                                           2   
South of Bloor                                                                4   
        Albanian Mosque - Albanian Muslim Society of To...                   10   
                       Baitul Aman Mosque                                     1   
                       Bosnian Mosque Birmingham                             33   
                       Hamza Mosque                                           3   
                       Omar Bin Khattab Mosque                                3   
                       Toronto Mosque                                         2   
Thistletown            Vaughn Islamic Community Centre Mosque                     
Underground city       Abu Huraira Mosque Inc (Abu Huraira Center)                
Upper Rouge            Nugget mosque                                              

                                                                          M_Lng  \
                                                                          count   
Neighborhoods          Mosques                                                    
North Midtown                                                                 3   
         Hamidiye Mosque                                                     31   
Railway Lands          Ajyal Mosque | مسجد أجيال (Ajyal Mosque)               1   
Royal York South West                                                         1   
 Baitul Hamd Mosque                                                           3   
South Niagara          Mosque Aisha                                           2   
South of Bloor                                                                4   
        Albanian Mosque - Alban

### 4. Data Visualisation

In [29]:
from bokeh.io import output_notebook, show
from bokeh.models import ColumnDataSource
from bokeh.palettes import Spectral5, Spectral9, Category20
from bokeh.plotting import figure
from bokeh.transform import factor_cmap
#from bokeh.resources import INLINE
#output_notebook(resources=INLINE)
output_notebook()

Loading BokehJS ...

##### How many mosque(s) within 5 kilometers of the neighborhood from each borough ?

In [33]:
group = mosque_count.groupby(by=['Boroughs','Neighborhoods'])

index_cmap = factor_cmap('Boroughs_Neighborhoods', palette=Spectral5, factors=sorted(mosque_count.Boroughs.unique()), end=1)

m = figure(plot_width=800, title="No. of Mosque by Neighborhood and Borough",
           x_range=group, toolbar_location=None, tooltips=[("NoofMosque", "@NoofMosque_mean")])

m.vbar(x='Boroughs_Neighborhoods', top='NoofMosque_mean', width=1, source=group,
       line_color="white", fill_color=index_cmap)

m.y_range.start = 0
m.x_range.range_padding = 0.05
m.xgrid.grid_line_color = None
m.xaxis.axis_label = "Neighborhood, Borough"
m.yaxis.axis_label = "No. of Mosque"
m.xaxis.major_label_orientation = 1.57
m.outline_line_color = None

show(m)

##### How many halal restaurants within 500 meters from each mosque?

In [31]:
#grp = halal_count.groupby(by=['Boroughs','Mosques'])
#nbhd = list(group.Neighborhoods)
#source = ColumnDataSource(data=dict(group = grp,neighborhood = nbhd))

group = halal_count.groupby(by=['Neighborhoods','Mosques'])
#print('{}'.format(group.shape()))
index_cmap = factor_cmap('Neighborhoods_Mosques', palette=Category20[15], factors=sorted(halal_count.Neighborhoods.unique()), end=1)

h = figure( title="No. of Halal Restaurant by Mosque and Borough",
           x_range=group, toolbar_location=None, tooltips=[("NoofHalalRestaurant", "@NoofHalalRestaurant_mean")])

h.vbar(x='Neighborhoods_Mosques', top='NoofHalalRestaurant_mean', width=1, source=group,
       line_color="white", fill_color=index_cmap)
#print('{}'.format(group[1]['Neigborhoods']))
h.y_range.start = 0
h.x_range.range_padding = 0.05
h.xgrid.grid_line_color = None
h.xaxis.axis_label = "Mosque, Neighborhood"
h.yaxis.axis_label = "No. of Restaurant Nearby Mosques"
h.xaxis.major_label_orientation = 1.57
h.xaxis.group_label_orientation =1.57
h.outline_line_color = None

show(h)

##### How many markets within 500 meters from each mosques?

In [32]:
group = market_count.groupby(by=['Neighborhoods'])

index_cmap = factor_cmap('Neighborhoods', palette=Spectral9, factors=sorted(market_count.Neighborhoods.unique()), end=1)

mr = figure( title="No. of Markets Nearby Mosque",
           x_range=group, toolbar_location=None, tooltips=[("NoofMarket", "@NoofMarket_mean")])

mr.vbar(x='Neighborhoods', top='NoofMarket_mean', width=1, source=group,
       line_color="white", fill_color=index_cmap)

mr.y_range.start = 0
mr.x_range.range_padding = 0.05
mr.xgrid.grid_line_color = None
mr.xaxis.axis_label = "Neighborhood"
mr.yaxis.axis_label = "No. of Market within 500m from Mosques"
mr.xaxis.major_label_orientation ='vertical'
mr.outline_line_color = None

show(mr)

In [ ]:
from IPython.display import HTML
import base64 

def create_download_link( df, title = "Download CSV file", filename = "x.csv"):  
    csv = x.to_csv()
    b64 = base64.b64encode(csv.encode())
    payload = b64.decode()
    html = '<a download="{filename}" href="data:text/csv;base64,{payload}" target="_blank">{title}</a>'
    html = html.format(payload=payload,title=title,filename=filename)
    return HTML(html)

create_download_link(df)